In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset=dataset.astype(int)

In [6]:
dataset

,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [7]:
dataset=dataset.drop("User ID",axis=1)

In [8]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [9]:
indep=dataset[["Age","EstimatedSalary","Gender_Male"]]
dep=dataset["Purchased"]

In [10]:
indep.shape

(400, 3)

In [11]:
dep

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    0
399    1
Name: Purchased, Length: 400, dtype: int32

In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(indep,dep,test_size=1/3,random_state=0)

In [13]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [14]:
from sklearn.linear_model import LogisticRegression

In [15]:
from sklearn.model_selection import GridSearchCV

param_grid=[{'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']},
              {'penalty':['none', 'elasticnet', 'l1', 'l2']},
              {'C':[0.001, 0.01, 0.1, 1, 10, 100]}]


grid=GridSearchCV(LogisticRegression(),param_grid, refit=True, verbose=3,n_jobs=-1,scoring='f1_weighted')

grid.fit(x_train,y_train)

Fitting 5 folds for each of 15 candidates, totalling 75 fits


C:\ANACONDA\Lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
10 fits failed out of a total of 75.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ANACONDA\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ANACONDA\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1162, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ANACONDA\Lib\site-packages\sklearn\linear_model\_logistic.py", line 54, in _check_solver
    raise Value

GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid=[{'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                     'saga']},
                         {'penalty': ['none', 'elasticnet', 'l1', 'l2']},
                         {'C': [0.001, 0.01, 0.1, 1, 10, 100]}],
             scoring='f1_weighted', verbose=3)

In [16]:
re=grid.cv_results_

grid_predictions=grid.predict(x_test)

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)


from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predictions)

In [25]:
table=pd.DataFrame.from_dict(re)

In [26]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_solver,param_penalty,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.019548,0.007111,0.002410,0.003093,newton-cg,NaN,NaN,{'solver': 'newton-cg'},0.799620,0.811321,0.682050,0.923510,0.922185,0.827737,0.089864,5
1,0.013548,0.009348,0.006435,0.001347,lbfgs,NaN,NaN,{'solver': 'lbfgs'},0.799620,0.811321,0.682050,0.923510,0.922185,0.827737,0.089864,5
2,0.004932,0.001627,0.006186,0.001742,liblinear,NaN,NaN,{'solver': 'liblinear'},0.804764,0.811321,0.682050,0.923510,0.922185,0.828766,0.089565,1
3,0.006194,0.002966,0.006116,0.002376,sag,NaN,NaN,{'solver': 'sag'},0.799620,0.811321,0.682050,0.923510,0.922185,0.827737,0.089864,5
4,0.004285,0.001852,0.005138,0.003259,saga,NaN,NaN,{'solver': 'saga'},0.799620,0.811321,0.682050,0.923510,0.922185,0.827737,0.089864,5
5,0.017602,0.005218,0.005304,0.003079,NaN,none,NaN,{'penalty': 'none'},0.804764,0.811321,0.682050,0.923510,0.922185,0.828766,0.089565,1
6,0.001891,0.000859,0.000000,0.000000,NaN,elasticnet,NaN,{'penalty': 'elasticnet'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
7,0.001537,0.000425,0.000000,0.000000,NaN,l1,NaN,{'penalty': 'l1'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
8,0.007244,0.002232,0.003784,0.003777,NaN,l2,NaN,{'penalty': 'l2'},0.799620,0.811321,0.682050,0.923510,0.922185,0.827737,0.089864,5
9,0.010154,0.004843,0.003720,0.002687,NaN,NaN,0.001,{'C': 0.001},0.509779,0.525300,0.501410,0.501410,0.501410,0.507862,0.009302,13


In [24]:
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_solver,param_penalty,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.019548,0.007111,0.002410,0.003093,newton-cg,NaN,NaN,{'solver': 'newton-cg'},0.799620,0.811321,0.682050,0.923510,0.922185,0.827737,0.089864,5
1,0.013548,0.009348,0.006435,0.001347,lbfgs,NaN,NaN,{'solver': 'lbfgs'},0.799620,0.811321,0.682050,0.923510,0.922185,0.827737,0.089864,5
2,0.004932,0.001627,0.006186,0.001742,liblinear,NaN,NaN,{'solver': 'liblinear'},0.804764,0.811321,0.682050,0.923510,0.922185,0.828766,0.089565,1
3,0.006194,0.002966,0.006116,0.002376,sag,NaN,NaN,{'solver': 'sag'},0.799620,0.811321,0.682050,0.923510,0.922185,0.827737,0.089864,5
4,0.004285,0.001852,0.005138,0.003259,saga,NaN,NaN,{'solver': 'saga'},0.799620,0.811321,0.682050,0.923510,0.922185,0.827737,0.089864,5
5,0.017602,0.005218,0.005304,0.003079,NaN,none,NaN,{'penalty': 'none'},0.804764,0.811321,0.682050,0.923510,0.922185,0.828766,0.089565,1
6,0.001891,0.000859,0.000000,0.000000,NaN,elasticnet,NaN,{'penalty': 'elasticnet'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
7,0.001537,0.000425,0.000000,0.000000,NaN,l1,NaN,{'penalty': 'l1'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,14
8,0.007244,0.002232,0.003784,0.003777,NaN,l2,NaN,{'penalty': 'l2'},0.799620,0.811321,0.682050,0.923510,0.922185,0.827737,0.089864,5
9,0.010154,0.004843,0.003720,0.002687,NaN,NaN,0.001,{'C': 0.001},0.509779,0.525300,0.501410,0.501410,0.501410,0.507862,0.009302,13


In [17]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter{}".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter{'solver': 'liblinear'} 0.8794289739855382


In [18]:
print("The confusion matrix:\n",cm)

The confusion matrix:
 [[79  6]
 [10 39]]


In [19]:
print("the report:\n",clf_report)

the report:
               precision    recall  f1-score   support

           0       0.89      0.93      0.91        85
           1       0.87      0.80      0.83        49

    accuracy                           0.88       134
   macro avg       0.88      0.86      0.87       134
weighted avg       0.88      0.88      0.88       134



In [20]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9507803121248499